# [DeepSphere]: a spherical convolutional neural network
[DeepSphere]: https://github.com/SwissDataScienceCenter/DeepSphere

[Nathanaël Perraudin](https://perraudin.info), [Michaël Defferrard](http://deff.ch), Tomasz Kacprzak, Raphael Sgier

# Figure: comparison of mass maps

In [ ]:
import os; 
import healpy as hp; import numpy as np; 
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:100% !important; }</style>"))
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
pl = plt

In [ ]:
pathfig = './figures/'
os.makedirs(pathfig, exist_ok=True)

In [ ]:
def make_zoom(map_test1, cmap=cm.gray_r, title='', rot = (0,45,-20), dtheta=5, clim = [-0.025, 0.041]):
    edge_arcmin = 2*dtheta*60
    n_pix = 500
    reso = edge_arcmin/float(n_pix)
    hp.gnomview(map_test1, reso=reso, xsize=n_pix, notext=True, title=title, rot=(0,0,0), cmap=cmap, min=clim[0], max=clim[1], hold=True)
    hp.graticule();

In [ ]:
def make_ball(map_test1, cmap=cm.gray_r, clim=None, title='', rot = (0,45,-20), dtheta = 5):
    cmap.set_over('w')
    cmap.set_under('w')
    cmap.set_bad('lightgrey')
    dot_size=10
    if clim==None:
        hp.visufunc.orthview(map=map_test1, half_sky=True, title=title, rot=rot, cmap=cmap, cbar=True, hold=True, nest=False, notext=True);
    else:
        hp.visufunc.orthview(map=map_test1, half_sky=True, title=title, rot=rot, cmap=cmap, cbar=True, hold=True, nest=False, notext=True, min=clim[0], max=clim[1]);
    theta, phi = hp.pix2ang(hp.npix2nside(len(map_test1)), range(len(map_test1)), nest=True);
#     hp.projscatter(theta, phi, c='k', s=dot_size);
    
    hp.graticule();
    hp.graticule(dmer=360,dpar=360,alpha=1, rot=(0,0,180-rot[2]), local=True);
    hp.graticule(dmer=360,dpar=360,alpha=1, rot=(0,0,-1*rot[2]), local=True);
        
    lw=2
    
    theta = np.linspace(90-dtheta, 90+dtheta, 100)*np.pi/180.
    phi = np.linspace(-dtheta, -dtheta, 100)*np.pi/180.
    hp.projplot(theta, phi, c='k', lw=lw);

    theta = np.linspace(90-dtheta, 90+dtheta, 100)*np.pi/180.
    phi = np.linspace(dtheta, dtheta, 100)*np.pi/180.
    hp.projplot(theta, phi, c='k', lw=lw);

    theta = np.linspace(90-dtheta, 90-dtheta, 100)*np.pi/180.
    phi = np.linspace(-dtheta, dtheta, 100)*np.pi/180.
    hp.projplot(theta, phi, c='k', lw=lw);

    theta = np.linspace(90+dtheta, 90+dtheta, 100)*np.pi/180.
    phi = np.linspace(-dtheta, dtheta, 100)*np.pi/180.
    hp.projplot(theta, phi, c='k', lw=lw);

In [ ]:
# map_kappa1 = np.load('./data/data_v5/kappa_omega_m_0p31_s_1.npy')
# map_kappa1 -= np.mean(map_kappa1)
# map_kappa2 = np.load('./data/data_v5/kappa_omega_m_0p26_s_1.npy')
# map_kappa2 -= np.mean(map_kappa2)

map_kappa1 = hp.read_map('data/same_psd/kappa_omega_m_0p31_s_2.fits')
map_kappa2 = hp.read_map('data/same_psd/kappa_omega_m_0p26_s_2.fits')

In [ ]:
sigma_hires, sigma_lores = 5, 60

In [ ]:
map_kappa1_lores = hp.ud_grade(map_kappa1, nside_out=512)
sigma_rad = sigma_lores / 60. * np.pi / 180.
map_kappa1_lores = hp.smoothing(map_kappa1_lores, fwhm=sigma_rad)

map_kappa1_hires = hp.ud_grade(map_kappa1, nside_out=1024)
sigma_rad = sigma_hires / 60. * np.pi / 180.
map_kappa1_hires = hp.smoothing(map_kappa1_hires, fwhm=sigma_rad)

In [ ]:
map_kappa2_lores = hp.ud_grade(map_kappa2, nside_out=512)
sigma_rad = sigma_lores / 60. * np.pi / 180.
map_kappa2_lores = hp.smoothing(map_kappa1_lores, fwhm=sigma_rad)

map_kappa2_hires = hp.ud_grade(map_kappa2, nside_out=1024)
sigma_rad = sigma_hires / 60. * np.pi / 180.
map_kappa2_hires = hp.smoothing(map_kappa2_hires, fwhm=sigma_rad)

In [ ]:
map_kappa1_hires.min(), map_kappa1_hires.max()

In [ ]:
map_kappa2_hires.min(), map_kappa2_hires.max()

In [ ]:
clim = [-0.014, 0.029]
clim_hires = [-0.03, 0.04]
rot = (0,-20,-20) 
dtheta = 5

In [ ]:
nx, ny, fs = 2, 2, 3
pl.figure(figsize=(nx*fs, ny*fs))
make_ball(map_kappa1_lores, cmap=pl.cm.plasma, clim=clim, rot=rot, dtheta=dtheta, title=r'Model 1: $\Omega_m=0.31$ $\sigma_8=0.82$' + '\nsmoothing 1 deg')
filename_plot = os.path.join(pathfig,'figure_kappa_diff.part1.svg')
pl.savefig(filename_plot, bbox_inches='tight', transparent=True); print('saved {}'.format(filename_plot))

pl.figure(figsize=(nx*fs, ny*fs))
make_zoom(map_kappa1_hires, cmap=cm.plasma, clim=clim_hires, rot = rot, dtheta=dtheta, title=r'zoom $10 \times 10$ deg'+'\nsmoothing 5 arcmin')
filename_plot = os.path.join(pathfig,'figure_kappa_diff.part2.svg')
pl.savefig(filename_plot, bbox_inches='tight', transparent=True); print('saved {}'.format(filename_plot))

pl.figure(figsize=(nx*fs, ny*fs))
make_ball(map_kappa2_lores, cmap=pl.cm.plasma, clim=clim, rot=rot, dtheta=dtheta, title=r'Model 2: $\Omega_m=0.26$ $\sigma_8=0.91$' + '\nsmoothing 1 deg')
filename_plot = os.path.join(pathfig,'figure_kappa_diff.part3.svg')
pl.savefig(filename_plot, bbox_inches='tight', transparent=True); print('saved {}'.format(filename_plot))

pl.figure(figsize=(nx*fs, ny*fs))
make_zoom(map_kappa2_hires, cmap=cm.plasma, clim=clim_hires, rot = rot, dtheta=dtheta, title=r'zoom $10 \times 10$ deg'+'\nsmoothing 5 arcmin')
filename_plot = os.path.join(pathfig,'figure_kappa_diff.part4.svg')
pl.savefig(filename_plot, bbox_inches='tight', transparent=True); print('saved {}'.format(filename_plot))


In [ ]:
import svgutils.compose as sc
from IPython.display import SVG # /!\ note the 'SVG' function also in svgutils.compose

In [ ]:
lw=2

In [ ]:
svg_fig = sc.Figure("20cm", "10cm",  
                    sc.SVG(os.path.join(pathfig,"figure_kappa_diff.part1.svg")).scale(1).move(0,   0),
                    sc.SVG(os.path.join(pathfig,"figure_kappa_diff.part2.svg")).scale(1).move(350, 0),
                    sc.Line( ((191,116), (390, 40)), width=lw, color='black'),
                    sc.Line( ((206,144), (390, 300)), width=lw, color='black')
                   )
svg_fig.save(os.path.join(pathfig,"figure_kappa_diff_model1.svg"))
SVG(os.path.join(pathfig,'figure_kappa_diff_model1.svg'))


In [ ]:
svg_fig = sc.Figure("20cm", "10cm",  
                    sc.SVG(os.path.join(pathfig,"figure_kappa_diff.part3.svg")).scale(1).move(0,   0),
                    sc.SVG(os.path.join(pathfig,"figure_kappa_diff.part4.svg")).scale(1).move(350, 0),
                    sc.Line( ((191,116), (390, 40)), width=lw, color='black'),
                    sc.Line( ((206,144), (390, 300)), width=lw, color='black')
                   )
svg_fig.save(os.path.join(pathfig,"figure_kappa_diff_model2.svg"))
SVG(os.path.join(pathfig,'figure_kappa_diff_model2.svg'))